In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score

DATA_DIR = 'Data/'

In [2]:
student_test_logs = pd.read_pickle(DATA_DIR + 'student_test_logs')
student_train_logs = pd.read_pickle(DATA_DIR + 'student_train_logs')
print(student_train_logs.shape)
student_train_logs.head()

(251488, 71)


,ITEST_id,SY ASSISTments Usage,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,...,confidence(CONFUSED),confidence(FRUSTRATED),confidence(OFF TASK),confidence(GAMING),RES_BORED,RES_CONCENTRATING,RES_CONFUSED,RES_FRUSTRATED,RES_OFFTASK,RES_GAMING
184889,9,1,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,...,0.000000,0.000000,0.838710,0.008522,0.376427,0.320317,0.000000,0.000000,0.785585,0.000264
184890,9,1,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,...,0.000000,0.091463,0.000000,0.047821,0.156027,0.225154,0.000000,0.009561,0.000000,0.001483
184891,9,1,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,...,0.000000,0.091463,0.280702,0.047821,0.156027,0.665929,0.000000,0.009561,0.149121,0.001483
184892,9,1,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,...,0.000000,0.091463,0.600000,0.047821,0.156027,0.780156,0.000000,0.009561,0.468252,0.001483
184893,9,1,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,...,0.378151,0.000000,0.578947,0.186970,0.376427,0.195349,0.060808,0.000000,0.440265,0.005797


In [3]:
train_labels = pd.read_csv('Data/training_label.csv', index_col='ITEST_id').sort_index()
train_labels.drop_duplicates(subset=None, keep='first', inplace=True)
train_labels.head()

,SchoolId,AveCorrect,MCAS,isSTEM
ITEST_id,,,,
9,2,0.438492,32,1
27,1,0.348837,21,0
33,2,0.686391,52,0
35,2,0.379658,34,0
37,3,0.305785,-999,0


In [4]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, fixed_dim, output_dim, batch_size=1, n_layers=1):
        super(LSTMClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            dropout=0.25,
                            bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*2 + fixed_dim, output_dim)

    def init_hidden(self):
        h0 = Variable(torch.zeros(self.n_layers*2, self.batch_size, self.hidden_dim))
        c0 = Variable(torch.zeros(self.n_layers*2, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, actions, fixed):
        hidden_state = self.init_hidden()
        out, _ = self.lstm(actions, hidden_state)
        y = self.hidden2label(torch.cat([out[-1, :, :], fixed], dim=1))
        return y

In [5]:
class TrainingSet(Dataset):
    def __init__(self):
        self.actions = student_train_logs.drop(['AveKnow',
                                        'AveCarelessness',
                                        'AveCorrect',
                                        'NumActions',
                                        'AveResBored',
                                        'AveResEngcon',
                                        'AveResConf',
                                        'AveResFrust',
                                        'AveResOfftask',
                                        'AveResGaming'], axis=1)
        self.fixed = student_train_logs[['ITEST_id',
                                        'AveKnow',
                                        'AveCarelessness',
                                        'AveCorrect',
                                        'NumActions',
                                        'AveResBored',
                                        'AveResEngcon',
                                        'AveResConf',
                                        'AveResFrust',
                                        'AveResOfftask',
                                        'AveResGaming']]
        self.y = train_labels
        
        self.weights = [0.5, 0.5]
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, id):
        studentId = self.y.iloc[id].name
        actions = self.actions[self.actions['ITEST_id'] == studentId].as_matrix().astype(np.float32)
        fixed = self.fixed[self.fixed['ITEST_id'] == studentId].as_matrix().astype(np.float32)[0]
        target = np.asarray([self.y.loc[studentId].isSTEM]).astype(np.float32)
        
        return torch.from_numpy(actions), torch.from_numpy(fixed), torch.from_numpy(target)
    
train_dataset = TrainingSet()
sampler = WeightedRandomSampler(train_dataset.weights, num_samples=len(train_dataset))
train_loader = DataLoader(train_dataset, sampler=sampler, batch_size=1, num_workers=4)

n_hidden = 64
lstm = LSTMClassifier(train_dataset.actions.shape[1] - 1,
                      n_hidden,
                      train_dataset.fixed.shape[1]-1,
                      output_dim=1,
                      batch_size=1,
                      n_layers=2)

criterion = nn.BCEWithLogitsLoss()
learning_rate = 0.0001 # If you set this too high, it might explode. If too low, it might not learn
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

losses = []
accs = []
bar1 = tqdm_notebook(range(10))
for epoch in bar1:
    train_loss = 0
    train_acc = []
    
    for i, (actions, fixed, target) in enumerate(tqdm_notebook(train_loader)):
        actions = Variable(actions[:, :, 1:]).permute(1,0,2)
        fixed = Variable(fixed[:, 1:])
        target = Variable(target)
        
        lstm.zero_grad()
        output = lstm(actions, fixed)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0]
        
        _, argmax = output.data.max(1)
        y_preds = argmax.view(-1).numpy()
        y_true = target.data.view(-1).numpy()
        train_acc.append(accuracy_score(y_true, y_preds))
    
    train_loss /= i+1
    
    losses.append(train_loss)
    accs.append(np.mean(train_acc))
    
    bar1.set_postfix(loss=losses[-1], acc=accs[-1])

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, max=467), HTML(value='')))

HBox(children=(IntProgress(value=0, max=467), HTML(value='')))

HBox(children=(IntProgress(value=0, max=467), HTML(value='')))

HBox(children=(IntProgress(value=0, max=467), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=467), HTML(value='')))

HBox(children=(IntProgress(value=0, max=467), HTML(value='')))

Process Process-26:
Process Process-28:
Process Process-27:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-25:
  File "/home/dario/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/dario/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/dario/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dario/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/dario/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dario/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/dario/anaconda3/lib/python3.6/site-packages/torch/uti

KeyboardInterrupt: 

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/dario/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/dario/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/dario/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.plot(accs)
plt.show()